In [1]:
import sys
sys.path.append("../")

In [2]:
from datetime import datetime, timedelta
from functools import partial
from matplotlib.dates import DateFormatter
from ta import add_all_ta_features
from ta.utils import dropna
from utils.common_train_utils import *
from utils.draw_candle_image import *
from utils.evaluate_old_models import *
import json
import keras
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import pathlib
import shutil
import ta
import ta.momentum
import ta.trend
import tempfile
import tensorflow as tf
import tensorflow_docs as tfdocs
import tensorflow_docs.modeling
import tensorflow_docs.plots
keras.config.enable_unsafe_deserialization()
from draw_image_utils import draw_prediction

In [ ]:
symbol_mapping = {
    "CTG": {
        "dataset_folder": "ctg_with_ema_macd_trend",
        "model_name": "model_5_bi2LSTM_ema_macd_trend",
        "freq_of_date": "B"
    },
    "FPT": {
        "dataset_folder": "fpt_with_ema_macd_trend",
        "model_name": "model_5_bi2LSTM_ema_macd_trend",
        "freq_of_date": "B"
    },
    "TCB": {
        "dataset_folder": "tcb_with_ema_macd_trend",
        "model_name": "model_5_bi2LSTM_ema_macd_trend",
        "freq_of_date": "B"
    },
    "VN30": {
        "dataset_folder": "vn30_with_ema_macd_trend",
        "model_name": "model_5_bi2LSTM_ema_macd_trend_1_1_1",
        "freq_of_date": "B"
    },
    # "BTC": {
    #     "dataset_folder": "btc_with_ema_macd_trend",
    #     "model_name": "model_5_biLSTM_ema_macd_trend",
    #     "freq_of_date": "D"
    # },
    "BTC": {
        "dataset_folder": "btc_with_ema_macd_trend",
        "model_name": "model_5_biLSTM_ema_macd_trend_1_1_1_1_1_1",
        "freq_of_date": "D"
    },
    "COCA_COLA": {
        "dataset_folder": "coca_cola_with_ema_macd_trend",
        "model_name": "model_5_bi2LSTM_ema_macd_trend_1_1_1",
        "freq_of_date": "D"
    },
}

In [4]:
day_previous_from_now = int(input("Day previous from now = ").strip())
now = datetime.now() - timedelta(days=day_previous_from_now)
date_continue_learning = str(now.date()).replace("-", "_")

In [5]:

symbol = input("Symbol = ").strip().upper()
dataset_folder_name = symbol_mapping[symbol]["dataset_folder"]
model_name = symbol_mapping[symbol]["model_name"]
dataset_folder = f"../dataset/{dataset_folder_name}"
to_folder = f"to_{date_continue_learning}"
continue_learning_folder = f"{dataset_folder}/continue_learning/{to_folder}"
index_of_image_input = 3
batch_size = 100
freq_of_date = symbol_mapping[symbol]["freq_of_date"] # B or D

config = read_config(dataset_folder)
candle_type_and_directory_save: dict = config["candle_type_and_directory_save"]
previous_days = config["previous_days"]
next_days = config["next_days"]
days_result = config["days_result"]


In [6]:
candle_type_and_directory_save

{'Is_start_bullish_engulfing_candle': 'bullish_engulfing',
 'Is_start_bearish_engulfing_candle': 'bearish_engulfing',
 'Is_star_doji_candle': 'star_doji',
 'Is_dragonfly_doji_candle': 'dragonfly_doji',
 'Is_gravestone_doji_candle': 'gravestone_doji',
 'Is_start_morning_star_candle': 'morning_star',
 'Is_start_evening_star_candle': 'evening_star',
 'Is_hammer_candle': 'hammer',
 'Is_inverted_hammer_candle': 'inverted_hammer',
 'Is_start_harami_down_candle': 'harami_down',
 'Is_start_harami_up_candle': 'harami_up',
 'Is_start_tweezer_top_candle': 'tweezer_top',
 'Is_start_tweezer_bottom_candle': 'tweezer_bottom',
 'Is_marubozu_candle': 'marubozu',
 'Is_spin_candle': 'spin',
 'Is_start_next_3_day_up_candle': 'next_3_day_up',
 'Is_start_next_3_day_down_candle': 'next_3_day_down'}

In [7]:
dataset_folder

'../dataset/btc_dataset_0_2_3_2014_2024_test_2024_2024_with_ema_macd_trend'

In [8]:
continue_learning_folder

'../dataset/btc_dataset_0_2_3_2014_2024_test_2024_2024_with_ema_macd_trend/continue_learning/to_2024_11_27'

In [ ]:
dataset = tf.data.Dataset.load(f'{continue_learning_folder}/test')

In [9]:
dataset_train = load_dataset_of_each_type_and_combine(f"{dataset_folder}/train", candle_type_and_directory_save, print_log=True)

folder: bullish_engulfing, 		 total: 79 images
folder: bearish_engulfing, 		 total: 80 images
folder: star_doji, 		 total: 25 images
folder: dragonfly_doji, 		 total: 7 images
folder: gravestone_doji, 		 total: 12 images
folder: morning_star, 		 total: 68 images
folder: evening_star, 		 total: 66 images
folder: hammer, 		 total: 22 images
folder: inverted_hammer, 		 total: 6 images
folder: harami_down, 		 total: 509 images
folder: harami_up, 		 total: 447 images
folder: tweezer_top, 		 total: 55 images
folder: tweezer_bottom, 		 total: 50 images
folder: marubozu, 		 total: 47 images
folder: spin, 		 total: 124 images
folder: next_3_day_up, 		 total: 475 images
folder: next_3_day_down, 		 total: 325 images
